# <center> TP2 - 01 Description

# Objective of TP

In this TP you will develop the full **supervised learning pipeline** including the *hyper-parameter tuning* and *model evalutaion*.  

You will then apply the pipeline to three algorithms
* nearest neighbour
* decision tree
* default classifier

Finally, you will perform *model comparison* and **discuss** its results.

### Recommendation:
The code you will develop in this TP is to be re-used in TP3 and the exam.  
Therefore we recommend you try to make it clear (use comments, when printing say what you print) so that next time it is easier for you to remember what it does.  
Also, try to make the code generic so that it can be easilly used for different datasets.   
Try to automate as much as possible so that the code does not require too much of your attention, finally you will need to do the same type of analysis not for 3 algorithms but for 5-6.

## Dataset

You will be workig with the same cars dataset as in TP1.  
Each group shall be using the same `brands` as in TP1.


In [1]:
# Load dataset and extract our part
import pandas as pd

# Reading csv file
autos = pd.read_csv('autos.csv',encoding='latin-1')

# Extracting the relevant part for our group
only_specific_brands = autos.brand.isin(['renault', 'peugeot', 'skoda', 'citroen', 'ford'])
autos = autos[only_specific_brands]

In [2]:
autos.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage,fast_sale
2,11400.0,limousine,2010.0,manuell,175.0,mondeo,125000.0,diesel,ford,nein,False
4,4100.0,kleinwagen,2009.0,manuell,68.0,1_reihe,90000.0,benzin,peugeot,nein,False
6,888.0,kombi,2000.0,manuell,115.0,mondeo,150000.0,benzin,ford,nein,True
7,13700.0,bus,2012.0,manuell,86.0,roomster,5000.0,benzin,skoda,nein,True
9,4299.0,kleinwagen,2010.0,manuell,75.0,2_reihe,125000.0,benzin,peugeot,nein,False


# Data preprocessing

Remember that after loading the dataset, there are several preprocessing steps you need to do before trainign the algorithm.
If you are not sure what these are, see *Course 8 - 02 Hyper-parameter tuning*.

When writing the code, **put short comments explaining what the pre-processing steps are and why you need to do them**.

### First we are changing the column names:

- We changed the style of writing the column names from camelcase to snakecase. This change will make column names easier to read, and we won't have to remember which letter is capitalized.
- We reworded some of the column names in order for them to be more descriptive and obvious.

In [3]:
# Changing the column names
autos.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 
                 'kilometer', 'fuel_type', 'brand', 'unrepaired_damage', 'fast_sale']
autos.head()

,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage,fast_sale
2,11400.0,limousine,2010.0,manuell,175.0,mondeo,125000.0,diesel,ford,nein,False
4,4100.0,kleinwagen,2009.0,manuell,68.0,1_reihe,90000.0,benzin,peugeot,nein,False
6,888.0,kombi,2000.0,manuell,115.0,mondeo,150000.0,benzin,ford,nein,True
7,13700.0,bus,2012.0,manuell,86.0,roomster,5000.0,benzin,skoda,nein,True
9,4299.0,kleinwagen,2010.0,manuell,75.0,2_reihe,125000.0,benzin,peugeot,nein,False


### Next, we are going to convert some columns data type:

- We are changing data type in order to facilitate the data processing (classifying the data in numeric or categorical)
- We are converting column 'unrepaired_damage' from object to boolean type
- We are converting column 'registration_year' from float to int.

In [4]:
# Converting column 'unrepaired_damage' from object to boolean type
autos['unrepaired_damage'] = (autos['unrepaired_damage']
                 .str.replace('nein','')
                 .str.replace('ja','True')
                  .astype(bool)
                 )

In [5]:
# Converting column 'registration_year' from float to int.
autos['registration_year'] = (autos['registration_year'].astype(int))

# Is it relevant or we can simply keep it in float format ?

In [6]:
autos.info()
autos.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28401 entries, 2 to 166073
Data columns (total 11 columns):
price                28401 non-null float64
vehicle_type         28401 non-null object
registration_year    28401 non-null int64
gearbox              28401 non-null object
power_ps             28401 non-null float64
model                28401 non-null object
kilometer            28401 non-null float64
fuel_type            28401 non-null object
brand                28401 non-null object
unrepaired_damage    28401 non-null bool
fast_sale            28401 non-null bool
dtypes: bool(2), float64(3), int64(1), object(5)
memory usage: 2.2+ MB


,price,registration_year,power_ps,kilometer
count,28401.000000,28401.000000,28401.000000,28401.000000
mean,4177.515017,2003.965565,102.402979,121472.307313
std,4680.629533,5.872460,40.664873,39816.529262
min,1.000000,1923.000000,2.000000,5000.000000
25%,1199.000000,2001.000000,75.000000,100000.000000
50%,2500.000000,2004.000000,101.000000,150000.000000
75%,5400.000000,2008.000000,122.000000,150000.000000
max,73500.000000,2016.000000,952.000000,150000.000000


### Next, we are goint to decide what is output (target) and what is input:
- We do this because ...
- How we do it ...

In [7]:
# output dataframe
out_autos = autos['fast_sale']
# input dataframe
in_autos = autos.iloc[:,[0,1,2,3,4,5,6,7,8,9]]

print('Inputs')
display(in_autos.head())
print('Outputs')
display(out_autos.head())

Inputs


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False


Outputs


2    False
4    False
6     True
7     True
9    False
Name: fast_sale, dtype: bool

### Next, we decide which is numerical and which is categorical data and how to treat in the following

- We do this because ...
- How we do it ...

In [8]:
print(in_autos.dtypes)
# DONE IN PREVIOUS STEP
# CHECK IF TYPES ARE CORRECT

price                float64
vehicle_type          object
registration_year      int64
gearbox               object
power_ps             float64
model                 object
kilometer            float64
fuel_type             object
brand                 object
unrepaired_damage       bool
dtype: object


### Next, we convert categorical to numeric (one-hot encoding)

- The algorithms we use (except the deciscion tree) cannot operate on label data directly. They require all input variables and output variables to be numeric. Therefore we need to convert categorical data to a numerical form.
- Since no ordinal relationship exists in the categorical variables in our dataset (and we don't want to allow the model to assume a natural ordering between categories), we use the one-hot encoding. 

In [9]:
print('Original input data')
print('-------------------')
display(in_autos.head())

# Create one-hot encoding
in_long = pd.get_dummies(in_autos)
print('\n Numerical input data')
print('-------------------')
display(in_long.head())

Original input data
-------------------


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False



 Numerical input data
-------------------


,price,registration_year,power_ps,kilometer,unrepaired_damage,vehicle_type_andere,vehicle_type_bus,vehicle_type_cabrio,vehicle_type_coupe,vehicle_type_kleinwagen,...,fuel_type_cng,fuel_type_diesel,fuel_type_elektro,fuel_type_hybrid,fuel_type_lpg,brand_citroen,brand_ford,brand_peugeot,brand_renault,brand_skoda
2,11400.0,2010,175.0,125000.0,False,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,4100.0,2009,68.0,90000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,888.0,2000,115.0,150000.0,False,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,13700.0,2012,86.0,5000.0,False,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,4299.0,2010,75.0,125000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### Next, we normalize (bring all variables to same scale [0,1])

-  We need to normalize the data to ensure that all sources are treated equally, and that data-availability bias (and its corresponding misrepresentation of the data universe) is reduced.

In [10]:
# maximum of each attribute
maxs = in_long.max(axis=0) # max accross rows (axis=0)

print('Maximums')
print('----------')
print(maxs.head())

# minimum of each attribute
mins = in_long.min(axis=0) # max accross rows (axis=0)

print('\n Minimums')
print('----------')
print(mins.head())

# min-max normalization
norm_in = (in_long - mins) / (maxs-mins)

# new maximum of each attribute
maxs = norm_in.max(axis=0) # max accross rows (axis=0)

print('\n New maximums')
print('----------')
print(maxs.head())

# new minimum of each attribute
mins = norm_in.min(axis=0) # max accross rows (axis=0)

print('\n New minimums')
print('----------')
print(mins.head())

Maximums
----------
price                 73500.0
registration_year      2016.0
power_ps                952.0
kilometer            150000.0
unrepaired_damage         1.0
dtype: float64

 Minimums
----------
price                   1.0
registration_year    1923.0
power_ps                2.0
kilometer            5000.0
unrepaired_damage       0.0
dtype: float64

 New maximums
----------
price                1.0
registration_year    1.0
power_ps             1.0
kilometer            1.0
unrepaired_damage    1.0
dtype: float64

 New minimums
----------
price                0.0
registration_year    0.0
power_ps             0.0
kilometer            0.0
unrepaired_damage    0.0
dtype: float64


### As of this step, no more use of original data and only use preprocessed

In our case above: `out_autos` and `norm_in`

# Prepare for model evaluation and hyper-parameter tuning

### Data splits for model evaluation (training and testing)

You will need to write the code splitting the data to training set (used for model learning and hyper-parameter tuning) and test set used for final model evaluation (test error).

Here, you can choose to **use either 5-folds cross-validation or 5 time repeated hold-out method.**

**Tell us what your choice is and why**. Both choices are good, we just want to know that you understand the differences and you have thought about them.

Remember that in the end this procedure will be used for all your algorithms and that these should work over the same train/test splits. You can make sure this will be the case by fixing the seed for the random sample generation.

### 5-folds cross-validation
- We chose cross-validation because it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split (hold-out).
- The more folds we have, the more we will be reducing the error due the bias but increasing the error due to variance; the computational price would go up too, obviously — the more folds you have, the longer it would take to compute it and you would need more memory. With a lower number of folds, we’re reducing the error due to variance, but the error due to bias would be bigger. It’s would also computationally cheaper

In [11]:
from sklearn.model_selection import KFold

def cross_validation(norm_in, out_autos,
                     hyper_parameters, callback_classifer, 
                     n_splits = 5, inner_splite = 3,
                     random_state = 123):
    hp_associative = {}
    trains = []
    tests = []
    for x in hyper_parameters:
        hp_associative[x] = 0    
    k_fold = KFold(n_splits = n_splits, random_state = random_state, shuffle = True)
    inner_k_fold = KFold(n_splits = inner_splite, random_state = random_state, shuffle = True)
    for train_idx,test_idx in k_fold.split(norm_in,out_autos):
        inner_cross_validation(inner_k_fold, train_idx, hp_associative, callback_classifer)    
        trains.append(train_idx)
        tests.append(test_idx)
    calculate_average_accuracy_hps(hp_associative, (n_splits * inner_splite))
    return hp_associative, trains, tests

In [12]:
def calculate_average_accuracy_hps(hp_associative, total_sum):
    for hp in hp_associative:
        hp_associative[hp] = hp_associative[hp] / total_sum
    return hp_associative

# Data splits for hyper-parameter tuning
Here we want you to use **3-folds inner cross validation.**

You will need to write the code to split each of the training sets above to train/validation accoridng to the 3-fold cross-validation strategy.

### Inner cross validation 


In [13]:
def inner_cross_validation(k_fold, train_idx, hp_associative, callback_classifer):
    for train_idx, validation_idx in k_fold.split(train_idx):
            in_train = norm_in.iloc[train_idx]
            out_train = out_autos.iloc[train_idx]
            in_validation = norm_in.iloc[validation_idx]
            out_validation = out_autos.iloc[validation_idx]
            calculate_accuracy_hp(hp_associative, callback_classifer, 
                         out_train, out_validation,
                         in_train, in_validation)

In [14]:
def calculate_accuracy_hp(hp_associative, callback_classifer, 
                         out_train, out_validation,
                         in_train, in_validation):
    for hp in hp_associative:
                accuracy = callback_classifer(out_train,out_validation,in_train,in_validation,hp)
                hp_associative[hp] =  hp_associative[hp] + accuracy 

### Generalization accuracy

You will also need to prepere the code that will use the trained models to produce predictions for the test instances, calculate the accuracy of over each test set, and calculate the final average accuracy over all the test instances (estimate of generalization accuracy).

In [15]:
def best_hyper_parameter(hyper_parameters):
    max = -1
    hp = -1
    for key, average in hyper_parameters.items():
        if (max < average):
            max = average
            hp = key
    return hp

def general_precision(callback_classifer, hyper_parameter,
                     trains, tests):
    x = 0 
    for train,test in zip(trains,tests):
        in_train = norm_in.iloc[train]
        out_train = out_autos.iloc[train]
        in_test= norm_in.iloc[test]
        out_test = out_autos.iloc[test]
        x += callback_classifer(out_train,out_test,in_train,in_test,hyper_parameter)
    return x / len(list(zip(trains,tests)))

# Train and test nearest neigbour model

Once you have the general procedure in place, train the nearest neigbour model.

### Hyper-parameter search

Hyper-parameter in nearest neighbour algorithm is the number of neighbours to use.
We want you to try at least 5 different values. **Tell us which values you decide to try.** (There is no Why questoin here.)

Remember that for choosing the best hyper-parameter value, you use the the inner cross validation and the best hyper-parameter is the one with the highest average accuracy over the validation sets.

### Model lerning and test accuracy

Once you have the best value of the hyper-parameter, you use it to **retrain** the model over the merged train+validation (you do this 5 times, see above *Data splits for model evaluation*). You then use this **retrained** model to get the final test accuracy.

For each of the test samples (there should be 5, see above), report the test accuracy and the corresponding hyper-parameter setting (the one chosen as best for this specific split).

Are the hyper-parameter parameters the same for all the test sets? **Discuss** if you think this is  normal or not, why it happens and if it creates some difficulties for interpreting the model. **There is no single correct answer here!** We want to see that you undertand the procedure and that you use your brain.



In [16]:
def calculate_pred_accuracy(pred_data, out_data):
    match_test = (pred_data == out_data)
    accuracy = match_test.sum() / match_test.count()
    return accuracy

def calculate_average_accuracy(couples_of_train_and_test_accuracies):
    train_acc = 0
    test_acc = 0
    size = len(couples_of_train_and_test_accuracies)
    for acc_couple in couples_of_train_and_test_accuracies:
        train_acc += acc_couple[0]
        test_acc += acc_couple[1]
    return (train_acc/size, test_acc/size)
def calculate_average_array_assoc(array_assoc):
    x = 0 
    for key in array_assoc:
        x += array_assoc[key]
    return x / len(array_assoc)

### Comments
- the best hyper-parameter found is 10

Are the hyper-parameter parameters the same for all the test sets?

In our case not because .... (to be developed)(I said "no.." because we use the method cross-validation)

In [17]:
from sklearn import neighbors

def nearest_neigbour(out_train,out_validation, in_train, in_validation, k = 5):
    nn_classifier = neighbors.KNeighborsClassifier(n_neighbors = k)
    # learn model using the training set inputs and outputs
    nn_classifier.fit(in_train,out_train)
    pred_test = nn_classifier.predict(in_validation)
    test_accuracy = calculate_pred_accuracy(pred_test, out_validation)
    return test_accuracy

# the nearest_neigbour is too long (it's normal), which is why I reduced the list of hyper-parametre
# If we want to test with more hyper-parameters, we have to reduce the split.
# execution time ≈ 3 min (It depends on the computing power of the compute)
averages_hp, trains, tests = cross_validation(norm_in, out_autos,
                                                  [5,10], nearest_neigbour) 
hyper_parameter = best_hyper_parameter(averages_hp)
accuracy_test = general_precision(nearest_neigbour, hyper_parameter,
                 trains, tests)
accuracy_train = calculate_average_array_assoc(averages_hp)
print('The best hyper-parameter is ',hyper_parameter)
print('Train accuracy ',accuracy_train)
print('Test accuracy ',accuracy_test)

The best hyper-parameter is  10
Train accuracy  0.5417016060547915
Test accuracy  0.5425866800875665


### Comment
- We decided to use xxx values
- We found that the best hyper-parameter value is yyy, because it has the highest average accuracy over the validation sets.

In [18]:
# 1. Find best hyper-parameter to use
# 2. Retrain the model over the merged train+validation 5 times
# 3. Use the retrained model to get the final test accuracy 
# 4. For each of the test samples (5), report the test accuracy and the corresponding hyper-parameter
# 5. Discuss and show that we understand the procedure

In [19]:
## 1. Find best hyper-parameter to use
#
#from sklearn import neighbors
#from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
#
## try K=1 through K=20 and record testing accuracy
#k_range = range(1, 25)
#
## keeping the accuracy scores
#scores = []
#
## We use a loop through the range 1 to 26
## We append the scores in the dictionary
#for k in k_range:
#    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
#    knn.fit(in_train,out_train)
#    y_pred = knn.predict(in_test)
#    scores.append(metrics.accuracy_score(out_test, y_pred))
#
#print(scores)

In [20]:
## import Matplotlib (scientific plotting library)
#import matplotlib.pyplot as plt
#
## allow plots to appear within the notebook
#%matplotlib inline
#
## plot the relationship between K and testing accuracy
## plt.plot(x_axis, y_axis)
#plt.plot(k_range, scores)
#plt.xlabel('Value of K for KNN')
#plt.ylabel('Testing Accuracy')
#
#max(scores)
#best_n_neighbors = scores.index(max(scores))
#print(best_n_neighbors)

- Training accuracy rises as model complexity increases
- Testing accuracy penalizes models that are too complex or not complex enough
- Above a K of 25 testing accuracy drops sharply
- We found the best value to be K=22

In [21]:
## 2. Retrain the model over the merged train+validation 5 times
#
#from sklearn import neighbors
#
#
#k = best_n_neighbors
#
## creaate nearest neighbour classifier object
#nn_classifier = neighbors.KNeighborsClassifier(n_neighbors=k)
#
## learn model using the training set inputs and outputs
#nn_classifier.fit(in_train,out_train)

In [22]:
## 3. Use the retrained model to get the final test accuracy 
#
## prediction for all train data
#preds_train = nn_classifier.predict(in_train)
#print('Prediction for train data')
#print('------------------------')
#display(preds_train)
#
## prediction for all test data
#preds_test = nn_classifier.predict(in_test)
#print('\n Prediction for test data')
#print('------------------------')
#display(preds_test)
#
## get test set accuracy
#match_preds = preds_test==out_test
#accuracy_test = match_preds.sum()/match_preds.count()
#print('Test accuracy', accuracy_test)
#
## get train set accuracy
#match_preds = preds_train==out_train
#accuracy_train = match_preds.sum()/match_preds.count()
#print('Train accuracy', accuracy_train)

# Train and test decision tree

Use the same general procedure to train a decision tree.

Hyper-parameters for decision trees are the pre-prunning criteria such as maximum number of leafs (see *Course 5 - 02 Decision tree prunning*). 
Pick one of these and use at least 5 different values. **Tell us which one you pick and what values you are using.**

Calculate and report the test accuracies together with their corresponding heper-parameter values. (No more comments needed here.)

In [23]:
from sklearn import tree

def deciscion_tree_classifier(out_train,out_validation, in_train, in_validation, max_leaf_nodes=15):
    # instantiate decition tree classifier
    dt_clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    # learn tree on train data
    dt_clf = dt_clf.fit(in_train,out_train)

    # make predictions for test set
    pred_test =  dt_clf.predict(in_validation)
    
    # calculate test accuracys
    test_accuracy = calculate_pred_accuracy(pred_test, out_validation)

    return test_accuracy

In [24]:
averages_hp, trains, tests = cross_validation(norm_in, out_autos,
                                                  [5,10,15,20,50], deciscion_tree_classifier)
hyper_parameter = best_hyper_parameter(averages_hp)
accuracy_test = general_precision(deciscion_tree_classifier, hyper_parameter,
                 trains, tests)
accuracy_train = calculate_average_array_assoc(averages_hp)
print('The best hyper-parameter is ',hyper_parameter)
print('Train accuracy ',accuracy_train)
print('Test accuracy ',accuracy_test)

The best hyper-parameter is  15
Train accuracy  0.5761131170151004
Test accuracy  0.5779023852674221


In [25]:
#import random
#
#
#random.seed(10000)
#dt_clf_accuracies = list()
#dt_clf = None
#
#for _ in range(0, 10):
#    out_train, out_test, in_train, in_test = kfold_spliting(norm_in, out_autos, 5, 1000)
#    train_acc, test_acc, dt_clf = deciscion_tree_classifier(out_train, out_test, in_train, in_test)
#    accuracy_couple = (train_acc, test_acc)
#    dt_clf_accuracies.append(accuracy_couple)

In [26]:
#rom sklearn.model_selection import KFold


#def kfold_spliting(input_data, output_data, n_splits, random_state, shuffle=True):
#   k_fold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True) 

#   for train_index,test_index in k_fold.split(norm_in,out_autos):
#       print(f'TRAIN: {train_index} | TEST: {train_index}')
#       in_train, in_test= norm_in.iloc[train_index], norm_in.iloc[test_index]
#       out_train, out_test = out_autos.iloc[train_index], out_autos.iloc[test_index]
#   
#   return (out_train, out_test, in_train, in_test)
#   

In [27]:
# Calculate average accuracy for Deciscion Tree
#train_accuracy, test_accuracy = calculate_average_accuracy(dt_clf_accuracies)
#
#print('Average train accuracy: ',train_accuracy)
#print('Average test accuracy: ',test_accuracy) 

# Train and test default classifier

Default classifier has no hyper-parameters, so you can skip the inner-cross validatoin procedure.

Calculate and report the test accuracies for the 5 test sets from the part *Data splits for model evaluation*

In [28]:
from sklearn.model_selection import train_test_split


def default_classifier(out_train, out_test):
    # get counts per class
    class_counts = out_train.value_counts()

    majority_class = class_counts.idxmax()

    # make predictions for train and test set
    pred_train = out_train.copy()
    pred_train[:] = majority_class

    pred_test = out_test.copy()
    pred_test[:] = majority_class

    # calculate train accuracy
    train_accuracy = calculate_pred_accuracy(pred_train, out_train)

    # calculate test accuracy
    test_accuracy = calculate_pred_accuracy(pred_test, out_test)
  
    return (train_accuracy, test_accuracy)

In [29]:
n_splits = 5
random_state = 123
default_clf_accuracies = list()
k_fold = KFold(n_splits = n_splits, random_state = random_state, shuffle = True)
for train_idx, test_idx in k_fold.split(norm_in,out_autos):
    #in_train = norm_in.iloc[train_idx]
    out_train = out_autos.iloc[train_idx]
    in_test = norm_in.iloc[test_idx]
    out_test = out_autos.iloc[test_idx]
    default_clf_accuracies.append(default_classifier(out_train, out_test))


In [30]:
# Calculate average accuracy for Default Dlassifer
train_accuracy, test_accuracy = calculate_average_accuracy(default_clf_accuracies)
print('Average train accuracy: ',train_accuracy)
print('Average test accuracy: ',test_accuracy) 

Average train accuracy:  0.5157564959759879
Average test accuracy:  0.5157566238834167


#  /!\ THIS IS THE MOST IMPORTANT PART /!\

# Compare models

Once you have all your test accuracies for the nearest neighbour, decision trees and default classifier, calculate the estimated generalization accuracy of each (the everage accuracy accross the test sets).

Is any of the algorithms peforming better than the other two? **Discuss, comment.**

## Use the McNemar test 

Use the McNemar test to verify whether the differences in the generalization accuracy are significant. 

In McNemar you can always compare only two algorithms. Do all the pair-wise comparisons, present and **explain** the results. Are these what you would expect?

In [31]:
def mcNemarTest(m1, m2, alpha = 0.05):
    ctab = pd.crosstab(m1,m2)
    #display(ctab)
    # calculate mcnemar test
    result = mcnemar(ctab)
    # interpret test results
    if result.pvalue < alpha:
        print('p-value < alpha: at {} significance level we reject H0. We conclude that M1 and M2 differ.'.format(alpha))
    else:
        print('p-value >= alpha: at {} significance level we cannot reject H0. We cannot say that M1 and M2 differ.'.format(alpha))

# Comparing the models

### Write an observation for each algorithm and compare performance
#### ==>
### Explain WHY some algorithms are better than the others
#### ==>

# The McNemar test

### Nearest neighbour vs Decision trees 
#### Result
#### Explaination
### Nearest neighbour vs Default classifier
#### Result
#### Explaination
### Decision trees vs Default classifier
#### Result
#### Explaination

 # Supervised learning pipeline with hyper-parameter tuning
 ### Automate as much as possible
 
#### Note: 1 and 2 are always somewhat different for new data set -> keep doing what we do
1. load data
2. data preprocessing
 * check and clean data (missing values, outliers) - not covered in DM course, hope you know from stats!
 * input vs output - decide what is output (target) and what input
 * numerical vs categorical data -  decide which is which and how to treat in the following
 * convert categorical to numeric (one-hot encoding)
 * normalization (bring all variables to same scale [0,1])
 * drop original data and keep working with these
 
========> DONE <========
 
#### Note: 3 to 7 are always the same procedures no matter what is the data set -> generalize the process 
3. prepare for training and testing
 * create train / test splits of data
 * remember one ranodm split not reliable if train and test sets not really big
 * use repated holdout, k-folds cross validation or repated bootstrap
4. prepare for hyper-parameter tuning
 * use inner cross-validation to split train set to train (for learning) and validation (for hyper-param seletction)
 * train and validate model for each hyper-parameter value
 * select best hyper-parameter (lowest error on average on validation sets)
5. train (learn) model $\mathbf{f} : \mathcal{X} \to \mathcal{Y}$  over train+validation sets using best hyper-param from step 3.
6. use the learned model $\mathbf{f}$ to do predictions
7. use test sets to get estimate of generalization error (accuracy)

Remember that steps 4.-7. need to be repeated for every train/test pslit from the outer cross-validation (out repated holdout splits)

#### Note: no idea yet
8. Try another model
 * eg. switch from decision trees to nearest neighbours
 * repeat the whole procedure
 * can you get better average test accuracy?
9. understand and describe models
 * how they work
 * how they predict
 * why one may be better than the other